In [17]:
cd learning/Resume Analyser/4.version/

C:\Users\Lenovo\learning\Resume Analyser\4.version


In [18]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 7010-C44B

 Directory of C:\Users\Lenovo\learning\Resume Analyser\4.version

11/20/2024  06:35 AM    <DIR>          .
11/04/2024  08:36 PM    <DIR>          ..
11/20/2024  06:34 AM    <DIR>          .ipynb_checkpoints
11/20/2024  06:33 AM    <DIR>          __pycache__
11/19/2024  02:12 PM             2,031 constants.py
11/20/2024  06:35 AM            54,962 funct_test.ipynb
11/11/2024  07:34 PM            23,719 HR_analytics.ipynb
11/11/2024  07:34 PM         5,213,294 job_final.csv
11/04/2024  08:38 PM    <DIR>          marker
11/19/2024  01:46 PM               411 prerequisite.py
11/19/2024  02:32 PM             3,673 pyresparser.py
11/19/2024  02:46 PM                98 requirements.txt
11/20/2024  06:34 AM                68 skills.csv
11/14/2024  10:13 PM    <DIR>          trainning
11/19/2024  02:29 PM            16,227 utils.py
               9 File(s)      5,314,483 bytes
               6 Dir(s)  202,382,499,840 bytes fre

In [19]:
pwd

'C:\\Users\\Lenovo\\learning\\Resume Analyser\\4.version'

In [20]:
!pip install -r requirements.txt
%run prerequisite.py

  Using cached spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached spacy-3.7.5-cp311-cp311-win_amd64.whl (12.1 MB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -------- ------------------------------- 2.1/10.0 MB 16.7 MB/s eta 0:00:01
   -------------------------------------- - 9.7/10.0 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 26.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -------------------------- ------------- 7.6/11.6 MB 36.2 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.6 MB 27.3 MB/s eta 0:00:01
   ------------------------------------- -- 11.0/11.6 MB 17.6 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 16.2 MB/s eta 0:00:00
Using cached thinc-8.2.5-cp311-cp311-win_amd64.whl (1.5 MB)
   ---------------------------------------- 0.0/2

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.5 which is incompatible.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roamin

In [3]:
import constants as cs

from spacy.matcher import Matcher
import spacy

import re
from collections import defaultdict
from urllib.parse import urlparse

import pandas as pd
import os

## Extract name

In [4]:
pattern = cs.NAME_PATTERN
pattern

[{'POS': 'PROPN'}]

In [5]:
def extract_name(doc, matcher):
    '''
    Helper function to extract name from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param matcher: object of `spacy.matcher.Matcher`
    :return: string of full name
    '''
    # Add a pattern to match proper nouns (PROPN)
    pattern = cs.NAME_PATTERN
    matcher.add('NAME', [pattern])

    # Apply the matcher to the doc
    matches = matcher(doc)

    # List to store full names
    full_names = []

    # Temporary variable to track the end position of the last matched name
    last_end_index = -1

    # Iterate over the matches
    for match_id, start, end in matches:
        # If the current match is contiguous with the last match, append the name
        if start == last_end_index:
            full_names[-1] += " " + doc[start:end].text  # Merge the names
        else:
            full_names.append(doc[start:end].text)  # Add a new name

        # Update the last_end_index to the end of the current match
        last_end_index = end

    return full_names


text = "John Doe is a software developer. And Scatter William Smith is an computer engineering."
nlp = spacy.load('en_core_web_sm')
matchers = Matcher(nlp.vocab)
full_names = extract_name(nlp(text), matcher=matchers)
print(full_names)

['John Doe', 'Scatter William Smith']


In [6]:
text = "John Doe is a software developer. And Scatter Will Smith is an computer engineering."
matchers = Matcher(nlp.vocab)
matchers.add('NAME', [[{'POS': 'PROPN'}]])
matchers(nlp(text))

[(424143666773229379, 0, 1),
 (424143666773229379, 1, 2),
 (424143666773229379, 8, 9),
 (424143666773229379, 10, 11)]

## Extract link

In [7]:
# Example text containing domain names with paths and some with protocols
text = """
Here are some domain names with and without protocols:
http://example.com
https://mywebsite.org/about
subdomain.example.net/products?category=books
another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file
"""

# Regular expression to match domain names with optional paths or query parameters
domain_with_path_pattern = r'\b(?:[A-Za-z][A-Za-z0-9+.-]*://)?[A-Za-z0-9.-]+\.[a-zA-Z]{2,}(?:/[A-Za-z0-9&%_\-.?=]*)*\b'

# Extracting all domain names with or without protocols
domains_with_paths = re.findall(domain_with_path_pattern, text)

# Prepare the final list with https:// if no protocol exists
final_links = []
for link in domains_with_paths:
    # If the link doesn't have a protocol (i.e., it doesn't start with 'http://', 'https://', etc.)
    if not re.match(r'^[a-zA-Z][a-zA-Z0-9+.-]*://', link):  # No protocol
        link = 'https://' + link  # Add https:// as the default protocol
    final_links.append(link)

# Print the final list of links
for link in final_links:
    print(link)


http://example.com
https://mywebsite.org/about
https://subdomain.example.net/products?category=books
https://another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file


In [8]:
def extract_links(text):
    # Regular expression to match domain names with optional protocols and paths
    domain_with_path_pattern = cs.LINK_PATTERN
    # Extract all the links (with or without protocol)
    links = re.findall(domain_with_path_pattern, text)

    # Dictionary to hold domain as key and list of links as value
    domain_dict = defaultdict(list)

    # Iterate through each link
    for link in links:
        # If the link doesn't have a protocol, add https:// by default
        if not re.match(r'^[a-zA-Z][a-zA-Z0-9+.-]*://', link):  # No protocol
            link = 'https://' + link

        # Parse the link to get the domain name (without protocol)
        parsed_url = urlparse(link)
        domain_name = parsed_url.netloc

        # Append the link to the list of the corresponding domain
        domain_dict[domain_name].append(link)

    return domain_dict

# Example text containing domain names with paths and some with protocols
text = """
Here are some domain names with and without protocols:
http://example.com
https://mywebsite.org/about
subdomain.example.net/products?category=books
another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file
"""

# Call the function and get the domain dictionary
domain_links = extract_links(text)

# Print the resulting dictionary
for domain, links in domain_links.items():
    print(f"{domain}:")
    for link in links:
        print(f"  {link}")


example.com:
  http://example.com
mywebsite.org:
  https://mywebsite.org/about
subdomain.example.net:
  https://subdomain.example.net/products?category=books
another-site.co.uk:
  https://another-site.co.uk/contact?ref=home
fileserver.com:
  ftp://fileserver.com/path/to/file


## Extract phone number

In [9]:
def extract_phone_numbers(text):
    # Regular expression to match phone numbers (supports various formats)
    phone_pattern = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{1,4}\)?[-.\s]?)?(\d{1,4})[-.\s]?(\d{1,4})[-.\s]?(\d{1,4})'

    # Find all matches in the text
    phone_numbers = re.findall(phone_pattern, text)

    # Clean up the result to return full phone numbers (combining the matched parts)
    full_phone_numbers = [''.join(number) for number in phone_numbers]

    return full_phone_numbers

# Example text with phone numbers
text = """
You can reach me at +1-800-555-5555 or (800) 555-5555.
My office number is 555 555 5555. Call me at +44 7911 123456.
Another contact: 123.456.7890. Or just use 800-123-4567.
"""

# Extract phone numbers from the text
phones = extract_phone_numbers(text)

# Print the extracted phone numbers
for phone in phones:
    print(phone)


+1-800-5555555
(800) 5555555
555 555 5555
+44 7911 123456
123.456.7890
800-123-4567


## Extract education

In [10]:
# Simulated constants
class cs:
    EDUCATION = {"BACHELOR", "MASTERS", "PHD", "MBA", "BSC", "MSC"}
    STOPWORDS = {"AND", "OF", "THE"}
    YEAR = r'\b(19|20)\d{2}\b'  # Regex pattern to match years like 1999, 2020

# Function to test
def extract_education(nlp_text):
    '''
    Helper function to extract education from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :return: tuple of education degree and year if year is found
             else only returns education degree
    '''
    edu = {}
    # Extract education degree
    try:
        for index, token in enumerate(nlp_text):
            text_cleaned = re.sub(r'[?|$|.|!|,]', r'', token.text)
            if text_cleaned.upper() in cs.EDUCATION and text_cleaned.upper() not in cs.STOPWORDS:
                # Collect surrounding context (current and next 5 tokens for simplicity)
                context = " ".join([t.text for t in nlp_text[index:index + 6]])
                edu[text_cleaned.upper()] = context
    except IndexError:
        pass

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(cs.YEAR), edu[key])
        if year:
            education.append((key, year.group(0)))  # Include degree and year
        else:
            education.append(key)  # Include degree only
    return education

# Test with spacy
nlp = spacy.load("en_core_web_sm")
test_text = "John completed his Bachelor of Science in 2015 and his Masters in 2020."
nlp_text = nlp(test_text)

# Execute the function
result = extract_education(nlp_text)
print(result)


[('BACHELOR', '2015'), ('MASTERS', '2020')]


## Extract skill

In [11]:
# Mock skills data
skills_data = {"Skill": ["python", "data analysis", "machine learning", "nlp", "deep learning"]}
skills_file_path = "skills.csv"

# Save the mock skills data to a CSV file
pd.DataFrame(skills_data).to_csv(skills_file_path, index=False)

# Function to extract skills
def extract_skills(nlp_text, noun_chunks, skills_file=None):
    '''
    Helper function to extract skills from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :param skills_file: path to skills file (optional)
    :return: list of skills extracted
    '''
    tokens = [token.text for token in nlp_text if not token.is_stop]
    if not skills_file:
        raise ValueError("Skills file is required.")
    else:
        data = pd.read_csv(skills_file)
    skills = list(data['Skill'].str.lower())  # Convert skills to lowercase for matching
    skillset = []
    # Check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    # Check for bi-grams and tri-grams
    for chunk in noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in skills:
            skillset.append(chunk_text)
    return [i.capitalize() for i in set(skillset)]

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Test text
test_text = "I have experience in Python, data analysis, and machine learning."
nlp_text = nlp(test_text)
noun_chunks = list(nlp_text.noun_chunks)

# Test the function
result = extract_skills(nlp_text, noun_chunks, skills_file_path)
print("Extracted Skills:", result)


Extracted Skills: ['Data analysis', 'Python', 'Machine learning', 'Python']


## Use model pre-train

In [23]:
pwd

'C:\\Users\\Lenovo'

In [24]:
model = spacy.load('learning/Resume Analyser/4.version/trainning/_model/output/model-best')

RegistryError: [E892] Unknown function registry: 'vectors'.

Available names: architectures, augmenters, batchers, callbacks, cli, datasets, displacy_colors, factories, initializers, languages, layers, lemmatizers, loggers, lookups, losses, misc, models, ops, optimizers, readers, schedules, scorers, tokenizers

In [ ]:
# Test example
main_str = ['''Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response
Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response''' ]


In [ ]:
for doc in model.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print((ent.text,ent.label_))
    # print([(ent.text, ent.label_) for ent in doc.ents])
